# マイクロマウスのターン生成

## 概要
マイクロマウス競技において、高速な走行を行うためにはスラロームと呼ばれる車体を止めずにターンをする動作が必要とされます。
このプログラムでは、スラローム走行に必要な軌跡を生成します。

## インプット
軌跡を作る際のインプットは以下になります。
    ターンの始点、終点、ターン角度、初期角度、曲線長、角速度変化調整用変数

以下のように直線と曲線を組み合わせてターンを生成します。

    直線 -> 曲線 -> 直線

曲線前後の直線の長さは曲線の長さに応じて自動で決定します。    

## アウトプット
スラロームの軌跡は弧長パラメータ付けされた曲率のリストで出力されます。
ロボットに実装する際には曲率のリストをロボットのプログラムにコピペし、軌道計画に利用してください。
軌道計画の方法についてはREADME.ipynbが参考になるかもしれません。

## マイクロマウス競技で用いるターン

### スラローム レベル 1 
まず、最低限のスラロームとして以下を実装しましょう。

 - 探索90度ターン
 - 最短90度ターン

### スラローム レベル 2
最短走行の際に半径の大きいターンを行うことでより高速で走行が可能です。

 - 大周り90度ターン
 - 180度ターン

### スラローム レベル 3
最短走行の際に区画を斜めに進むことで大幅な走行時間短縮が可能です。
ただし、壁にぶつからないように制御を行うことが少々難しいです。

 - 直進から斜め45度ターン
 - 斜めから直進45度ターン
 - 直進から斜め135度ターン
 - 斜めから直進135度ターン
 - 斜め90度ターン

このうち、45度ターンと135度ターンは直進から斜めと斜めから直進のパターンがありますが、軌跡を逆からたどれば一方が他方になるので、実質3つのターンになります。
### スラローム レベル 4
上記以外にも変則的なスラローム起動が存在します。
(こじまターン等と呼ばれるらしいです。)
    

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib.ticker as tick
import numpy as np 
from numba import jit
import math 
from itertools import product

from ipywidgets import interact, interactive, fixed




In [2]:
# molifireの積分値を計算するための関数


#y = math.e / a * np.exp( (-1)/(1- np.abs(2*x/b -1)**c))

@jit
def moli(x,a,b,c):
    return math.e / a * np.exp( (-1)/(1- np.abs(2*x/b -1)**c))

@jit
def int_moli(c):
    delta_t = 0.00001
    x = np.arange(0.000001,1,delta_t)

    S = 0
    y = moli(x,1,1,c)
    for i,ele in enumerate(y):
        if i != 0:
            S = S + delta_t * ele
    return S    


In [3]:

# 軌跡を作るためのクラス
class CalcTraj:
    shape_factor = 0
    path_length = 0
    target_ang = 0
    delta_t = 0
    tread = 0
    start_ang = 0
    start_x = 0
    start_y = 0
    
    s = None
    kappa =  None
    D_kappa = None
    I_kappa = None
    c_traj_x = None
    c_traj_y = None
    l_traj_x = None
    l_traj_y = None
    r_traj_x = None
    r_traj_y = None
    
    # 軌跡のパラメータをセットする
    def set_param(self, shape_factor_, path_length_, target_ang_, delta_t_, tread_, start_x_=0, start_y_=0, start_ang_=0):
        self.shape_factor = shape_factor_
        self.path_length = path_length_
        self.target_ang = target_ang_
        self.delta_t = delta_t_
        self.tread = tread_
        self.start_x = start_x_
        self.start_y = start_y_
        self.start_ang = start_ang_
        
    # 軌跡を計算する    
    @jit
    def calc_traj(self):
        self.s = np.arange(0.00001, self.path_length, self.delta_t)
        
        # a * b * S = target_ang
        c = self.shape_factor
        b = self.path_length
        S = int_moli(c)
        a = 1/(self.target_ang/b/S)
        
        # 曲率 
        self.kappa = np.zeros(len(self.s))
        for i in range(len(self.s)):
            val = 1 - np.abs(2*self.s[i]/b -1)**c
            if val != 0.0:
                self.kappa[i] = math.e / a * np.exp(-1/val) 
            else:
                self.kappa[i] = 0.0
        
        # 曲率の微分
        self.D_kappa = np.gradient(self.kappa)/ self.delta_t

        
        # 曲率の積分
        self.I_kappa = np.zeros(len(self.kappa))
        self.I_kappa[0] = self.start_ang;
        for i,ele in enumerate(self.kappa):
            if i != 0:
                self.I_kappa[i] = self.I_kappa[i-1] + self.delta_t * ele 

        # 軌跡のx座標, y座標を求める        
        self.c_traj_x = np.zeros(len(self.kappa))
        self.c_traj_y = np.zeros(len(self.kappa))

        self.l_traj_x = np.zeros(len(self.kappa))
        self.l_traj_y = np.zeros(len(self.kappa))

        self.r_traj_x = np.zeros(len(self.kappa))
        self.r_traj_y = np.zeros(len(self.kappa))

        self.c_traj_x[0] = self.start_x
        self.c_traj_y[0] = self.start_y
        
        self.r_traj_x[0] = self.c_traj_x[0] + math.sin(self.start_ang)*self.tread/2
        self.r_traj_y[0] = self.c_traj_y[0] - math.cos(self.start_ang)*self.tread/2

        self.l_traj_x[0] = self.c_traj_x[0] - math.sin(self.start_ang)*self.tread/2
        self.l_traj_y[0] = self.c_traj_y[0] + math.cos(self.start_ang)*self.tread/2
        
        for i,ele in enumerate(self.I_kappa):
            if i != 0:
                self.c_traj_x[i] = self.c_traj_x[i-1] + self.delta_t *  math.cos(ele)
                self.c_traj_y[i] = self.c_traj_y[i-1] + self.delta_t *  math.sin(ele)
                self.r_traj_x[i] = self.c_traj_x[i] + math.sin(ele)*self.tread/2
                self.r_traj_y[i] = self.c_traj_y[i] - math.cos(ele)*self.tread/2

                self.l_traj_x[i] = self.c_traj_x[i] - math.sin(ele)*self.tread/2
                self.l_traj_y[i] = self.c_traj_y[i] + math.cos(ele)*self.tread/2
        
            
    # 軌跡が終わったポイントのx座標
    def get_end_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1]
    # 軌跡が終わったポイントのy座標    
    def get_end_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1]
    
    # x方向の移動距離
    def get_move_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1] - self.start_x
    
    # y方向の移動距離
    def get_move_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1] - self.start_y
        
    # 速度一定で走ったときの時間と角速度のリストを返す
    def get_omega(self, v):
        return self.s /v, self.kappa * v
    
    # 速度一定で走ったときの時間と角速度(degree)のリストを返す
    def get_omega_degree(self, v):
        t, rad = self.get_omega(v) 
        return t, rad * 180 /math.pi
    
    # 速度一定で走ったときの時間と角加速度のリストを返す
    def get_alpha(self, v):
        t, rad = self.get_omega(v)
        alpha = np.gradient(rad)/ (self.delta_t/v)
        return t,alpha 

    # 速度一定で走ったときの時間と角加速度(degree)のリストを返す
    def get_alpha_degree(self, v):
        t, alpha = self.get_alpha(v)
        return t, alpha * 180/math.pi
    
    # 速度一定で走ったときの時間と向進加速度のリストを返す
    def get_cent_acc(self, v):
        return self.s/v, v * v * self.kappa 
        
    
    
    # 速度一定で走ったときに右タイヤにかかる力を返す
    def get_r_force(self, v, m, I):
        t, rad = self.get_omega(v)
        t, alpha = self.get_alpha(v)
        omega_f = m * v * 0.5 * rad
        alpha_f = I * alpha / (2 * self.tread)
        f = np.sqrt(omega_f * omega_f + alpha_f * alpha_f)
        return t, omega_f, alpha_f, f
    
    # 速度一定で走ったときに左タイヤにかかる力を返す
    def get_l_force(self, v, m, I):
        t, rad = self.get_omega(v)
        t, alpha = self.get_alpha(v)
        omega_f = m * v * 0.5 * rad
        alpha_f = I * alpha / (2 * self.tread)
        f =  np.sqrt(omega_f * omega_f + alpha_f * alpha_f)
        return t, omega_f, alpha_f, f
        

## ハーフ探索90度ターン
壁を読む関係上ターンの開始、終了を0.5cmずらしています。

In [13]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.005 -0.045
start_y = 0.00
end_x = 0.0
end_y = -0.005 + 0.045
target_ang = 90
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.06
v__ = 0.36
traj_hf_serch_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_hf_serch_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_pre = (end_x - start_x) - traj_curve.get_move_x()
    d_fol = (end_y - start_y) - traj_curve.get_move_y()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre +tread/2 , end_x+tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre -tread/2, end_x-tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 

    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [12]:
print(" ")
print("len(kappa) " + str(len(traj_hf_serch_90.kappa)))
print("s resolution[m] " + str(traj_hf_serch_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_serch_90.s, traj_cl_serch_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

NameError: name 'traj_cl_serch_90' is not defined

## ハーフ最短90度

In [18]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.045
start_y = 0.00
end_x = 0.0
end_y = 0.045
target_ang = 90
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.063
v__ = 0.5
traj_hf_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_hf_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_pre = (end_x - start_x) - traj_curve.get_move_x()
    d_fol = (end_y - start_y) - traj_curve.get_move_y()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre +tread/2 , end_x+tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre -tread/2, end_x-tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 

    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [17]:
print(" ")
print("len(kappa) " + str(len(traj_hf_90.kappa)))
print("s resolution[m] " + str(traj_hf_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_hf_90.s, traj_hf_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 6299
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
6.151625929900779e-262,
3.041182775718933e-174,
2.138238175156888e-130,
4.3476443710333054e-104,
1.5032947130370648e-86,
5.066920191972151e-74,
1.260412291212018e-64,
2.5605127776098203e-57,
1.7971043239443644e-51,
1.0910913193971926e-46,
1.056647214075976e-42,
2.4931798871301742e-39,
1.9397314849595547e-36,
6.212424623552019e-34,
9.673586190833841e-32,
8.317448327814912e-30,
4.359696364505524e-28,
1.5063407956850418e-26,
3.652085739262376e-25,
6.535636496526646e-24,
8.99800836482724e-23,
9.8621473036862e-22,
8.854057578592259e-21,
6.668950142182666e-20,
4.3004668868989095e-19,
2.4155274348973452e-18,
1.1994193322784417e-17,
5.3325036133044335e-17,
2.1463088229771128e-16,
7.896502104272856e-16,
2.6780382549930663e-15,
8.434374224601957e-15,
2.4830154958012185e-14,
6.872404469076469e-14,
1.797520357330663e-13,
4.463406454180781e-13,
1.0564978264476601e-12,
2.3926523170732332e-12,
5.2016213600655014e-12,
1.0887920609958405e

30.283429570860413,
30.29566849010581,
30.307891885089443,
30.3200997705172,
30.33229216108127,
30.344469071460136,
30.35663051631859,
30.36877651030768,
30.380907068064786,
30.393022204213533,
30.40512193336385,
30.417206270111922,
30.42927522904024,
30.44132882471754,
30.453367071698842,
30.46538998452544,
30.477397577724876,
30.489389865810963,
30.50136686328379,
30.513328584629686,
30.525275044321255,
30.537206256817363,
30.549122236563104,
30.561022997989866,
30.57290855551526,
30.584778923543173,
30.596634116463726,
30.608474148653315,
30.620299034474563,
30.632108788276366,
30.643903424393866,
30.65568295714844,
30.667447400847745,
30.679196769785666,
30.690931078242347,
30.702650340484194,
30.714354570763852,
30.72604378332023,
30.7377179923785,
30.749377212150044,
30.761021456832577,
30.772650740609993,
30.7842650776525,
30.79586448211654,
30.807448968144826,
30.81901854986634,
30.8305732413963,
30.842113056836236,
30.853638010273915,
30.865148115783384,
30.876643387424973,
30

36.711964094799725,
36.714129300612115,
36.716288568010505,
36.71844190386533,
36.72058931504684,
36.72273080842509,
36.72486639087,
36.72699606925136,
36.72911985043885,
36.731237741302046,
36.73334974871047,
36.73545587953353,
36.73755614064067,
36.739650538901245,
36.74173908118466,
36.74382177436028,
36.74589862529757,
36.74796964086598,
36.750034827935075,
36.75209419337449,
36.754147744053995,
36.756195486843446,
36.75823742861286,
36.760273576232436,
36.762303936572536,
36.764328516503724,
36.7663473228968,
36.7683603626228,
36.770367642553005,
36.77236916955898,
36.77436495051257,
36.77635499228597,
36.77833930175169,
36.780317885782594,
36.78229075125192,
36.784257905033286,
36.78621935400074,
36.78817510502874,
36.790125164992226,
36.79206954076656,
36.79400823922762,
36.79594126725181,
36.79786863171602,
36.79979033949771,
36.8017063974749,
36.80361681252621,
36.80552159153083,
36.80742074136859,
36.80931426892,
36.811202181066186,
36.81308448468897,
36.8149611866709,
36.816

37.027956833245625,
37.0268049671083,
37.025648494376455,
37.02448740783916,
37.02332170028967,
37.02215136452532,
37.02097639334758,
37.019796779561936,
37.01861251597793,
37.017423595409085,
37.01623001067289,
37.0150317545908,
37.013828819988156,
37.012621199694166,
37.01140888654191,
37.010191873368264,
37.00897015301392,
37.007743718323304,
37.0065125621446,
37.00527667732968,
37.004036056734094,
37.00279069321704,
37.00154057964131,
37.00028570887331,
36.99902607378299,
36.99776166724386,
36.99649248213289,
36.995218511330556,
36.99393974772078,
36.99265618419088,
36.99136781363158,
36.99007462893697,
36.98877662300449,
36.98747378873486,
36.9861661190321,
36.98485360680346,
36.98353624495946,
36.982214026413786,
36.98088694408331,
36.97955499088804,
36.978218159751115,
36.97687644359876,
36.975529835360284,
36.974178327968005,
36.97282191435727,
36.97146058746641,
36.97009434023673,
36.96872316561245,
36.96734705654073,
36.965966005971566,
36.96458000685785,
36.963189052155286,


25.715682261351663,
25.69790135686067,
25.680099838340713,
25.66227768836399,
25.644434889506126,
25.626571424346146,
25.608687275466746,
25.590782425454407,
25.57285685689955,
25.55491055239668,
25.536943494544555,
25.518955665946354,
25.500947049209792,
25.48291762694735,
25.46486738177637,
25.446796296319228,
25.42870435320356,
25.410591535062306,
25.392457824533995,
25.374303204262844,
25.356127656898938,
25.33793116509839,
25.319713711523566,
25.30147527884316,
25.283215849732432,
25.26493540687341,
25.24663393295493,
25.22831141067302,
25.20996782273083,
25.19160315183904,
25.17321738071589,
25.15481049208741,
25.136382468687636,
25.117933293258698,
25.099462948551103,
25.08097141732386,
25.062458682344722,
25.04392472639026,
25.025369532246224,
25.00679308270758,
24.988195360578768,
24.96957634867393,
24.950936029816976,
24.93227438684196,
24.91359140259314,
24.894887059925203,
24.8761613417035,
24.857414230804256,
24.8386457101147,
24.819855762533326,
24.801044370970125,
24.782

## ハーフ大周り90度

In [20]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.09
start_y = 0.00
end_x = 0.0
end_y = 0.09
target_ang = 90
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.1
v__ = 0.5
traj_hf_L_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_hf_L_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_pre = (end_x - start_x) - traj_curve.get_move_x()
    d_fol = (end_y - start_y) - traj_curve.get_move_y()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre +tread/2 , end_x+tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre -tread/2, end_x-tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 

    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [21]:
print(" ")
print("len(kappa) " + str(len(traj_hf_90.kappa)))
print("s resolution[m] " + str(traj_hf_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_hf_L_90.s, traj_hf_L_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 6299
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
0.0,
1.890901227870531e-277,
7.501143281053391e-208,
4.3072116170270666e-166,
2.9755689760197827e-138,
2.284717146736787e-118,
1.874024386780439e-103,
7.455983746825776e-92,
1.4199907276513604e-82,
5.556870958590486e-75,
1.1801801334561897e-68,
2.6647780620731904e-63,
1.0340829627899286e-58,
9.80832834459455e-55,
2.9614379933807967e-51,
3.483433277363527e-48,
1.867855829814624e-45,
5.168672319274686e-43,
8.150973290644438e-41,
7.938185003995953e-39,
5.0986793461588444e-37,
2.280333165692597e-35,
7.43006326173814e-34,
1.8320861452361804e-32,
3.5304063030809873e-31,
5.4641936805148685e-30,
6.954207640004415e-29,
7.4264354959064405e-28,
6.772410021281656e-27,
5.355153751861861e-26,
3.721112838468878e-25,
2.2990502152187662e-24,
1.276147948725175e-23,
6.422708891014102e-23,
2.954913036737976e-22,
1.251819857033942e-21,
4.915166861938589e-21,
1.799171274705565e-20,
6.172047046959521e-20,
1.9937476730060016e-19,
6.0906103523344

12.132164861791159,
12.141991863272741,
12.151812737990875,
12.161627486164464,
12.171436108026858,
12.181238603825786,
12.191034973823308,
12.200825218295742,
12.210609337533601,
12.220387331841525,
12.230159201538216,
12.23992494695638,
12.249684568442655,
12.259438066357562,
12.269185441075415,
12.278926692984289,
12.288661822485937,
12.298390829995741,
12.308113715942634,
12.317830480769047,
12.327541124930852,
12.337245648897293,
12.34694405315091,
12.356636338187528,
12.366322504516132,
12.376002552658838,
12.385676483150851,
12.395344296540372,
12.405005993388558,
12.414661574269433,
12.424311039769888,
12.433954390489557,
12.443591627040792,
12.453222750048587,
12.462847760150568,
12.472466657996847,
12.482079444250044,
12.491686119585196,
12.501286684689683,
12.510881140263217,
12.520469487017746,
12.530051725677398,
12.539627856978447,
12.549197881669235,
12.558761800510132,
12.568319614273468,
12.577871323743476,
12.587416929716259,
12.5969564329997,
12.60648983441343,
12.61

17.0215158492316,
17.027960488782952,
17.034400314761335,
17.040835329891216,
17.04726553689646,
17.053690938500385,
17.060111537425694,
17.06652733639453,
17.072938338128427,
17.07934454534832,
17.085745960774545,
17.092142587126823,
17.098534427124267,
17.104921483485356,
17.111303758927964,
17.117681256169323,
17.124053977926042,
17.130421926914075,
17.136785105848748,
17.143143517444727,
17.149497164416033,
17.15584604947603,
17.162190175337393,
17.16852954471216,
17.174864160311692,
17.181194024846658,
17.18751914102705,
17.193839511562185,
17.200155139160675,
17.20646602653044,
17.212772176378692,
17.219073591411963,
17.22537027433605,
17.231662227856035,
17.237949454676304,
17.244231957500492,
17.250509739031532,
17.256782801971603,
17.263051149022164,
17.269314782883935,
17.275573706256857,
17.28182792184017,
17.28807743233232,
17.294322240431026,
17.30056234883322,
17.30679776023508,
17.313028477332004,
17.319254502818634,
17.325475839388805,
17.331692489735595,
17.33790445655

22.567603635655235,
22.56925179577282,
22.570897960625203,
22.57254213140938,
22.574184309321883,
22.57582449555876,
22.57746269131561,
22.579098897787546,
22.58073311616923,
22.58236534765484,
22.583995593438107,
22.58562385471229,
22.58725013267017,
22.588874428504095,
22.590496743405936,
22.592117078567103,
22.593735435178544,
22.59535181443076,
22.596966217513778,
22.598578645617184,
22.6001890999301,
22.601797581641193,
22.603404091938685,
22.60500863201033,
22.606611203043446,
22.608211806224887,
22.609810442741065,
22.61140711377794,
22.613001820521028,
22.61459456415538,
22.616185345865638,
22.61777416683596,
22.61936102825007,
22.620945931291267,
22.62252887714238,
22.624109866985822,
22.625688902003557,
22.627265983377086,
22.62884111228751,
22.630414289915464,
22.631985517441162,
22.63355479604437,
22.63512212690442,
22.636687511200222,
22.63825095011024,
22.63981244481252,
22.641371996484654,
22.64292960630383,
22.644485275446783,
22.646039005089836,
22.647590796408874,
22.

23.438704077429065,
23.438680576458594,
23.43865670020567,
23.438632446444267,
23.438607812957024,
23.438582797535172,
23.43855739797836,
23.438531612094586,
23.43850543770006,
23.43847887261912,
23.438451914684105,
23.438424561735264,
23.438396811620645,
23.43836866219601,
23.43834011132472,
23.438311156877663,
23.438281796733136,
23.438252028776773,
23.43822185090145,
23.438191261007187,
23.43816025700109,
23.438128836797237,
23.438096998316617,
23.438064739487032,
23.43803205824303,
23.43799895252584,
23.437965420283263,
23.43793145946962,
23.437897068045686,
23.437862243978593,
23.43782698524178,
23.437791289814918,
23.43775515568385,
23.4377185808405,
23.437681563282833,
23.4376441010148,
23.43760619204623,
23.4375678343928,
23.437529026075985,
23.43748976512298,
23.43745004956662,
23.437409877445386,
23.437369246803275,
23.437328155689798,
23.437286602159894,
23.4372445842739,
23.437202100097466,
23.437159147701546,
23.4371157251623,
23.437071830561084,
23.43702746198437,
23.4369

22.198283783672927,
22.196211809438015,
22.19413758849723,
22.19206111953923,
22.189982401252013,
22.18790143232293,
22.185818211438665,
22.183732737285226,
22.181645008547946,
22.179555023911536,
22.177462782059987,
22.17536828167665,
22.173271521444207,
22.17117250004465,
22.169071216159324,
22.166967668468896,
22.164861855653346,
22.16275377639198,
22.160643429363446,
22.158530813245715,
22.156415926716065,
22.1542987684511,
22.15217933712675,
22.150057631418267,
22.147933650000226,
22.145807391546494,
22.14367885473029,
22.141548038224133,
22.13941494069985,
22.1372795608286,
22.135141897280846,
22.133001948726353,
22.130859713834205,
22.128715191272814,
22.12656837970988,
22.124419277812418,
22.122267884246753,
22.12011419767851,
22.117958216772628,
22.115799940193344,
22.11363936660421,
22.11147649466807,
22.109311323047066,
22.107143850402657,
22.104974075395585,
22.10280199668591,
22.10062761293297,
22.098450922795408,
22.096271924931177,
22.0940906179975,
22.091907000650913,
2

15.632125770151976,
15.624657953711184,
15.617184770996511,
15.609706219319122,
15.602222295991394,
15.59473299832688,
15.587238323640378,
15.579738269247864,
15.572232832466634,
15.564722010615192,
15.557205801013307,
15.549684200982078,
15.542157207843864,
15.534624818922323,
15.527087031542479,
15.519543843030677,
15.511995250714605,
15.504441251923344,
15.496881843987303,
15.489317024238346,
15.481746790009717,
15.474171138636066,
15.466590067453513,
15.459003573799626,
15.451411655013393,
15.44381430843532,
15.436211531407418,
15.42860332127317,
15.420989675377594,
15.413370591067277,
15.405746065690302,
15.398116096596345,
15.390480681136669,
15.382839816664136,
15.375193500533209,
15.367541730099953,
15.359884502722123,
15.352221815759084,
15.344553666571906,
15.336880052523313,
15.329200970977768,
15.321516419301403,
15.313826394862122,
15.306130895029545,
15.298429917175083,
15.290723458671915,
15.283011516894986,
15.275294089221095,
15.267571173028834,
15.25984276569865,
15.2

5.355799012054093,
5.343377731144108,
5.330957176752363,
5.31853738457394,
5.306118390447061,
5.293700230353404,
5.281282940418567,
5.268866556912486,
5.256451116249719,
5.2440366549899915,
5.231623209838482,
5.219210817646304,
5.20679951541084,
5.194389340276116,
5.1819803295333315,
5.169572520621101,
5.157165951125939,
5.144760658782626,
5.132356681474648,
5.119954057234501,
5.107552824244174,
5.0951530208355065,
5.082754685490611,
5.07035785684223,
5.057962573674105,
5.045568874921484,
5.033176799671394,
5.020786387163097,
5.0083976767884435,
4.996010708092343,
4.9836255207730265,
4.971242154682542,
4.958860649827138,
4.9464810463675954,
4.934103384619668,
4.921727705054398,
4.909354048298634,
4.896982455135298,
4.884612966503815,
4.8722456235004925,
4.859880467378959,
4.847517539550419,
4.835156881584168,
4.822798535207932,
4.810442542308215,
4.798088944930731,
4.785737785280693,
4.773389105723358,
4.7610429487842305,
4.748699357149531,
4.7363583736665795,
4.724020041344128,
4.7116

## ハーフ180度ターン
180度ターンは曲線長と角度を指定するだけでうまく終点と始点が結べないため注意。
カーブの終点が180度ターンの終了点になるように曲線長を2分探索で求めている。

In [25]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.09
start_y = 0.00
end_x = -0.09
end_y = 0.09
target_ang = 180
start_ang = 0.0

shape_factor__ = 2.6

v__ = 0.5
traj_hf_180 = CalcTraj()
d_pre__ = 0.01

@interact(shape_factor=(0.1, 8.0, 0.1), d_pre=(0.001, 0.3, 0.001) ,  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, d_pre = d_pre__, v=v__):
    
    traj_curve = traj_hf_180
    path_length_min = 0.01
    path_length_max = 0.5
    error_y = 1.0
    
    # 最小化するターゲット関数
    def calc_y_error(pl_):
        traj = CalcTraj()
        traj.set_param(shape_factor_=shape_factor, path_length_= pl_, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)        
        traj.calc_traj()
        error_y_ = traj.get_move_y() - end_y
        return error_y_
    
    while abs(error_y) > 0.0001:
        pl = 0.5 * (path_length_min + path_length_max)
        error_y = calc_y_error(pl)
        path_length = pl
        if(error_y > 0):
            path_length_max = pl
        else:
            path_length_min = pl
        print("error_y: " + str(error_y))
        print("path_length: " + str(pl))
        print("min, max :",path_length_min, path_length_max)
        
    d_fol = d_pre
    
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)        
    traj_curve.calc_traj()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y()+tread/2, traj_curve.get_end_y()+tread/2]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), traj_curve.get_end_y()]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y()-tread/2, traj_curve.get_end_y()-tread/2]  ,color="red") 


    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

    print("move_y[m] " + str(traj_curve.get_move_y()) )  
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [26]:
print(" ")
print("len(kappa) " + str(len(traj_hf_180.kappa)))
print("s resolution[m] " + str(traj_hf_180.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_hf_180.s, traj_hf_180.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 19422
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
0.0,
0.0,
0.0,
0.0,
2.104333645261666e-269,
8.830545815868213e-231,
8.187290379248533e-202,
2.7742523985089427e-179,
2.9318985437902734e-161,
1.637038726143976e-146,
3.185292027268274e-134,
7.97276578792627e-124,
6.524891973027062e-115,
3.460519490526189e-107,
1.986722646672911e-100,
1.8279159317200483e-94,
3.6570232481661256e-89,
2.024325285825713e-84,
3.759384900180874e-80,
2.7377679832944073e-76,
8.882994336379458e-73,
1.4269307932627704e-69,
1.239059373790926e-66,
6.261602950986429e-64,
1.9602414676089755e-61,
4.009331483548902e-59,
5.607629448057577e-57,
5.578356460630175e-55,
4.083672141958313e-53,
2.2662182723970773e-51,
9.784449781587708e-50,
3.362524410256772e-48,
9.38498241619393e-47,
2.1656371417645322e-45,
4.1976553581173523e-44,
6.931657382414929e-43,
9.875762311084341e-42,
1.2278290965860263e-40,
1.3457873231075146e-39,
1.3124740040050235e-38,
1.1484283680343007e-37,
9.084523638104846e-37,
6.541432073449136

3.5806378491053907,
3.586894599874476,
3.593153542984136,
3.5994146698051606,
3.6056779717219425,
3.6119434401324435,
3.6182110664482083,
3.624480842094375,
3.6307527585096455,
3.637026807146281,
3.6433029794701506,
3.6495812669606624,
3.6558616611107992,
3.6621441534271124,
3.668428735429703,
3.6747153986522534,
3.6810041346419866,
3.687294934959678,
3.69358779117966,
3.6998826948897925,
3.706179637691513,
3.712478611199773,
3.718779607043054,
3.725082616863405,
3.7313876323163644,
3.7376946450710187,
3.744003646809967,
3.7503146292293237,
3.7566275840387267,
3.7629425029613075,
3.7692593777337087,
3.775578200106081,
3.781898961842056,
3.7882216547187615,
3.7945462705268067,
3.800872801070278,
3.8072012381667597,
3.813531573647283,
3.8198637993563453,
3.8261979071519083,
3.8325338889054064,
3.8388717365016976,
3.8452114418390875,
3.8515529968293243,
3.857896393397597,
3.86424162348251,
3.8705886790360844,
3.8769375520237634,
3.883288234424392,
3.8896407182302335,
3.8959949954469324,
3

9.969797908839109,
9.975745512390063,
9.981691585889255,
9.98763612854686,
9.99357913957586,
9.999520618192063,
10.005460563614099,
10.011398975063402,
10.017335851764196,
10.023271192943536,
10.029204997831222,
10.035137265659882,
10.04106799566489,
10.046997187084417,
10.052924839159399,
10.058850951133527,
10.064775522253244,
10.070698551767775,
10.076620038929041,
10.082539982991753,
10.088458383213315,
10.094375238853873,
10.100290549176323,
10.106204313446229,
10.112116530931903,
10.118027200904363,
10.12393632263729,
10.129843895407102,
10.135749918492884,
10.141654391176408,
10.147557312742133,
10.153458682477162,
10.159358499671301,
10.165256763616995,
10.171153473609346,
10.177048628946116,
10.182942228927695,
10.18883427285713,
10.19472476004009,
10.200613689784873,
10.2065010614024,
10.21238687420622,
10.218271127512464,
10.224153820639907,
10.2300349529099,
10.23591452364639,
10.24179253217592,
10.247668977827612,
10.25354385993318,
10.259417177826895,
10.265288930845603,


17.07624780502781,
17.07983749966428,
17.08342583048556,
17.08701279787806,
17.09059840222819,
17.09418264392233,
17.09776552334685,
17.101347040888125,
17.104927196932486,
17.10850599186628,
17.112083426075834,
17.11565949994745,
17.119234213867436,
17.122807568222054,
17.126379563397595,
17.129950199780303,
17.13351947775641,
17.137087397712147,
17.14065396003372,
17.144219165107334,
17.147783013319142,
17.151345505055332,
17.15490664070203,
17.158466420645382,
17.162024845271485,
17.165581914966438,
17.169137630116317,
17.17269199110719,
17.176244998325096,
17.179796652156043,
17.183346952986067,
17.186895901201133,
17.19044349718721,
17.19398974133026,
17.197534634016208,
17.201078175630965,
17.204620366560412,
17.208161207190436,
17.211700697906874,
17.21523883909557,
17.21877563114231,
17.222311074432916,
17.225845169353125,
17.22937791628869,
17.232909315625342,
17.236439367748776,
17.239968073044682,
17.2434954318987,
17.247021444696475,
17.250546111823624,
17.25406943366572,
1

21.130169983781272,
21.132112809046262,
21.13405476803052,
21.135995860996992,
21.137936088208523,
21.13987544992787,
21.141813946417717,
21.14375157794063,
21.145688344759122,
21.147624247135592,
21.149559285332362,
21.15149345961167,
21.15342677023565,
21.155359217466373,
21.157290801565797,
21.159221522795804,
21.161151381418193,
21.163080377694662,
21.16500851188683,
21.166935784256236,
21.168862195064307,
21.170787744572415,
21.172712433041813,
21.174636260733692,
21.176559227909127,
21.178481334829147,
21.180402581754652,
21.182322968946476,
21.18424249666535,
21.186161165171963,
21.188078974726846,
21.18999592559049,
21.19191201802329,
21.19382725228556,
21.1957416286375,
21.19765514733926,
21.19956780865088,
21.201479612832305,
21.20339056014341,
21.205300650843984,
21.207209885193723,
21.209118263452226,
21.211025785879027,
21.212932452733547,
21.214838264275144,
21.216743220763075,
21.21864732245652,
21.220550569614552,
21.222452962496185,
21.224354501360335,
21.2262551864658

...

18.776112934466855,
18.77318335139393,
18.770252599538352,
18.767320678544095,
18.764387588055065,
18.761453327715017,
18.75851789716769,
18.755581296056693,
18.75264352402555,
18.74970458071772,
18.746764465776547,
18.743823178845304,
18.740880719567173,
18.73793708758525,
18.73499228254254,
18.732046304081972,
18.729099151846363,
18.726150825478467,
18.72320132462095,
18.720250648916373,
18.717298798007217,
18.714345771535896,
18.71139156914471,
18.708436190475894,
18.70547963517157,
18.7025219028738,
18.699562993224557,
18.6966029058657,
18.693641640439026,
18.690679196586263,
18.68771557394901,
18.684750772168798,
18.68178479088709,
18.678817629745236,
18.675849288384523,
18.67287976644612,
18.669909063571144,
18.666937179400627,
18.663964113575478,
18.660989865736553,
18.658014435524617,
18.655037822580343,
18.652060026544323,
18.649081047057063,
18.64610088375898,
18.643119536290406,
18.64013700429161,
18.63715328740273,
18.634168385263862,
18.631182297514997,
18.628195023796042,

15.205141932438593,
15.200857946440438,
15.196572417140079,
15.192285344191314,
15.187996727248175,
15.183706565964853,
15.179414859995811,
15.175121608995736,
15.170826812619492,
15.166530470522229,
15.162232582359263,
15.157933147786153,
15.153632166458728,
15.149329638032956,
15.145025562165102,
15.140719938511648,
15.136412766729288,
15.132104046474938,
15.127793777405783,
15.123481959179188,
15.119168591452786,
15.114853673884461,
15.110537206132252,
15.10621918785451,
15.101899618709815,
15.097578498356913,
15.093255826454875,
15.088931602662942,
15.084605826640646,
15.08027849804772,
15.075949616544145,
15.071619181790156,
15.067287193446235,
15.062953651173064,
15.058618554631607,
15.054281903483087,
15.049943697388908,
15.045603936010792,
15.04126261901065,
15.036919746050673,
15.032575316793308,
15.02822933090121,
15.023881788037313,
15.01953268786483,
15.01518203004714,
15.01082981424796,
15.006476040131233,
15.002120707361138,
14.99776381560211,
14.99340536451888,
14.989045

10.111239033937018,
10.105326586804436,
10.099412592996037,
10.093497053243954,
10.087579968282984,
10.081661338850672,
10.075741165687294,
10.06981944953578,
10.063896191141884,
10.057971391254084,
10.052045050623535,
10.046117170004223,
10.040187750152874,
10.034256791828959,
10.028324295794715,
10.02239026281516,
10.016454693658106,
10.010517589094174,
10.004578949896691,
9.998638776841858,
9.992697070708694,
9.986753832278938,
9.980809062337213,
9.974862761670984,
9.968914931070463,
9.962965571328777,
9.957014683241841,
9.951062267608409,
9.945108325230152,
9.9391528569115,
9.933195863459817,
9.927237345685333,
9.921277304401118,
9.91531574042313,
9.909352654570245,
9.903388047664192,
9.897421920529629,
9.891454273994087,
9.885485108888028,
9.879514426044853,
9.87354222630081,
9.867568510495154,
9.861593279470052,
9.855616534070583,
9.849638275144809,
9.843658503543733,
9.837677220121288,
9.831694425734407,
9.825710121243013,
9.819724307509933,
9.813736985401036,
9.807748155785195,

3.730451605150356,
3.724146886881635,
3.717844175483288,
3.7115434793017927,
3.705244806697121,
3.698948166042771,
3.692653565725838,
3.6863610141468564,
3.680070519719994,
3.673782090872944,
3.6674957360469245,
3.6612114636967124,
3.654929282290677,
3.6486492003106914,
3.6423712262522328,
3.6360953686243622,
3.6298216359496185,
3.6235500367642097,
3.6172805796178573,
3.6110132730738655,
3.604748125709148,
3.5984851461141427,
3.5922243428929312,
3.585965724663122,
3.579709300055917,
3.573455077716129,
3.5672030663021683,
3.560953274485954,
3.554705710953091,
3.548460384402697,
3.542217303547546,
3.535976477113957,
3.5297379138418403,
3.523501622484732,
3.5172676118097628,
3.51103589059759,
3.5048064676425366,
3.4985793517525217,
3.492354551748974,
3.486132076467002,
3.479911934755284,
3.4736941354760718,
3.4674786875052224,
3.4612655997321635,
3.4550548810599406,
3.4488465404052095,
3.4426405866981176,
3.436437028882496,
3.430235875915736,
3.4240371367687863,
3.4178408204261808,
3.4116

1.52105956208734e-07,
1.372097436262899e-07,
1.2363731315935086e-07,
1.1128393294930624e-07,
1.0005203836788541e-07,
8.985082796181668e-08,
8.059587635396614e-08,
7.220876372335237e-08,
6.461672147681156e-08,
5.775229371974355e-08,
5.1553014126628365e-08,
4.5961097806285234e-08,
4.0923147753035e-08,
3.6389875470602016e-08,
3.231583535275275e-08,
2.865917240253959e-08,
2.5381382870739524e-08,
2.2447087392917607e-08,
1.9823816205292034e-08,
1.748180602001341e-08,
1.5393808142096732e-08,
1.353490741275377e-08,
1.1882351566538173e-08,
1.041539059363242e-08,
9.115125702741632e-09,
7.964367485015273e-09,
6.947502884921391e-09,
6.050370590152785e-09,
5.260144459072883e-09,
4.5652246115220025e-09,
3.955135816283736e-09,
3.4204328165699774e-09,
2.9526122433315322e-09,
2.544030774997168e-09,
2.1878292114008542e-09,
1.8778621391286446e-09,
1.6086328751709041e-09,
1.3752333857709765e-09,
1.173288887474862e-09,
9.989068476552584e-10,
8.486301122300996e-10,
7.193938987763251e-10,
6.084864037788983e-

## クラシック 斜め 45度ターン

In [33]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.09
start_y = 0.00
end_x = 0.0
end_y = 0.045
target_ang = 45
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.06
v__ = 0.5
traj_hf_45 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_hf_45
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_fol = ((end_y - start_y) - traj_curve.get_move_y()) * math.sqrt(2)
    d_pre = (end_x - start_x) - traj_curve.get_move_x() - d_fol/math.sqrt(2)
    
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.r_traj_x[-1] + d_pre , traj_curve.r_traj_x[-1] + d_pre + d_fol/math.sqrt(2) ],[traj_curve.r_traj_y[-1], traj_curve.r_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[-1] + d_pre , traj_curve.l_traj_x[-1] + d_pre + d_fol/math.sqrt(2) ],[traj_curve.l_traj_y[-1], traj_curve.l_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 

    
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [34]:
print(" ")
print("len(kappa) " + str(len(traj_hf_45.kappa)))
print("s resolution[m] " + str(traj_hf_45.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_hf_45.s, traj_hf_45.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 5999
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
1.0886094871283891e-249,
3.589343014189223e-166,
2.0609763341468106e-124,
2.341515936556653e-99,
1.1833256063761339e-82,
1.009821385931095e-70,
8.965883368978496e-62,
8.173606953828793e-55,
3.021794402346525e-49,
1.0858619527837802e-44,
6.792477742203701e-41,
1.1072321272587493e-37,
6.274196017561005e-35,
1.526738454363484e-32,
1.8693560161168214e-30,
1.3001028530379521e-28,
5.643675017734715e-27,
1.6472751690336953e-25,
3.43120916067649e-24,
5.352257409178421e-23,
6.50371687657427e-22,
6.360170039738426e-21,
5.1433725391329355e-20,
3.5188660290120864e-19,
2.0764081677861358e-18,
1.0742769927330024e-17,
4.9423320312803086e-17,
2.046606123578411e-16,
7.708939823529465e-16,
2.665597405124465e-15,
8.529385981514172e-15,
2.5434652314048048e-14,
7.112462781204747e-14,
1.8753960036993118e-13,
4.685667974910577e-13,
1.1141724848981091e-12,
2.5312376305170353e-12,
5.513610566439961e-12,
1.155134983438459e-11,
2.334317501341033e-1

14.73033185485428,
14.738594950083417,
14.746847563881149,
14.75508970642696,
14.763321387893876,
14.771542618448445,
14.779753408250695,
14.78795376745412,
14.796143706205635,
14.80432323464554,
14.812492362907506,
14.82065110111852,
14.828799459398903,
14.836937447862228,
14.845065076615324,
14.853182355758223,
14.861289295384157,
14.869385905579513,
14.877472196423813,
14.88554817798967,
14.893613860342793,
14.901669253541927,
14.909714367638841,
14.917749212678297,
14.925773798698032,
14.933788135728726,
14.941792233793974,
14.949786102910263,
14.957769753086952,
14.965743194326219,
14.97370643662309,
14.981659489965363,
14.989602364333619,
14.99753506970116,
15.005457616034022,
15.013370013290944,
15.021272271423324,
15.02916440037522,
15.037046410083317,
15.044918310476898,
15.052780111477823,
15.06063182300053,
15.06847345495199,
15.07630501723168,
15.084126519731585,
15.091937972336153,
15.099739384922298,
15.107530767359357,
15.115312129509077,
15.123083481225608,
15.130844832

19.12154942412578,
19.123144164566213,
19.124735007930713,
19.126321958477266,
19.12790502046251,
19.129484198141746,
19.13105949576893,
19.132630917596707,
19.134198467876416,
19.135762150858106,
19.137321970790506,
19.138877931921105,
19.140430038496106,
19.14197829476046,
19.143522704957878,
19.145063273330823,
19.146600004120558,
19.148132901567102,
19.14966196990931,
19.151187213384812,
19.152708636230077,
19.154226242680394,
19.155740036969906,
19.157250023331592,
19.158756205997303,
19.16025858919777,
19.1617571771626,
19.163251974120282,
19.16474298429824,
19.166230211922805,
19.167713661219214,
19.169193336411677,
19.170669241723328,
19.172141381376274,
19.173609759591592,
19.175074380589333,
19.176535248588557,
19.177992367807317,
19.17944574246268,
19.180895376770742,
19.182341274946648,
19.183783441204564,
19.18522187975775,
19.186656594818512,
19.18808759059824,
19.189514871307416,
19.19093844115564,
19.192358304351615,
19.193774465103164,
19.195186927617247,
19.1965956960

18.844610794275415,
18.842395923676992,
18.840176505187408,
18.83795253423179,
18.83572400623235,
18.83349091660842,
18.831253260776414,
18.829011034149836,
18.826764232139258,
18.824512850152306,
18.822256883593678,
18.81999632786509,
18.81773117836531,
18.815461430490117,
18.813187079632304,
18.81090812118167,
18.80862455052499,
18.806336363046036,
18.80404355412556,
18.80174611914125,
18.79944405346776,
18.797137352476685,
18.794826011536564,
18.792510026012827,
18.790189391267837,
18.78786410266085,
18.785534155548035,
18.783199545282404,
18.780860267213868,
18.778516316689192,
18.776167689051988,
18.773814379642722,
18.771456383798668,
18.769093696853943,
18.766726314139465,
18.76435423098296,
18.76197744270893,
18.759595944638665,
18.75720973209024,
18.75481880037847,
18.75242314481493,
18.750022760707935,
18.74761764336253,
18.74520778808048,
18.74279319016027,
18.740373844897064,
18.737949747582732,
18.735520893505832,
18.73308727795157,
18.730648896201824,
18.728205743535124,


13.405951464494047,
13.396034242029936,
13.386104985621738,
13.376163684702622,
13.366210328708908,
13.356244907080168,
13.346267409259307,
13.336277824692742,
13.326276142830409,
13.316262353125984,
13.306236445036902,
13.296198408024534,
13.286148231554264,
13.276085905095654,
13.26601141812246,
13.255924760112869,
13.245825920549557,
13.235714888919755,
13.2255916547155,
13.215456207433604,
13.205308536575908,
13.19514863164931,
13.184976482165936,
13.174792077643243,
13.16459540760415,
13.154386461577172,
13.144165229096522,
13.133931699702266,
13.12368586294043,
13.113427708363142,
13.103157225528744,
13.09287440400197,
13.082579233353988,
13.072271703162633,
13.06195180301248,
13.051619522494985,
13.041274851208646,
13.030917778759102,
13.020548294759319,
13.010166388829651,
12.999772050598086,
12.9893652697003,
12.978946035779812,
12.968514338488182,
12.95807016748506,
12.947613512438435,
12.937144363024698,
12.926662708928836,
12.91616853984454,
12.905661845474414,
12.895142615

0.05906271107666937,
0.0569425025982859,
0.05487556088841308,
0.052861264214334625,
0.05089898243408937,
0.04898807717731675,
0.0471279020351358,
0.045317802759072705,
0.043557117469173484,
0.04184517687134417,
0.04018130448399808,
0.03856481687404095,
0.0369950239022587,
0.035471228978116415,
0.033992729323988266,
0.032558816248825916,
0.031168775431226498,
0.02982188721191311,
0.028517426895527217,
0.02725466506172398,
0.026032867885445194,
0.024851297466288638,
0.023709212166868044,
0.022605866959984483,
0.021540513784495258,
0.020512401909652887,
0.019520778307738747,
0.01856488803474858,
0.01764397461886788,
0.0167572804564795,
0.01590404721536897,
0.015083516244818436,
0.014294928992214948,
0.013537527425791101,
0.012810554463077902,
0.01211325440462781,
0.011444873372524776,
0.01080465975318452,
0.010191864643905355,
0.009605742302596053,
0.00904555060010029,
0.008510551474467871,
0.008000011386536702,
0.007513201776116315,
0.007049399518068569,
0.006607887377530946,
0.006187954

## クラシック 斜め 135度ターン

In [40]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.09
start_y = 0.00
end_x = -0.045
end_y = 0.09
target_ang = 135
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.14
v__ = 0.5
traj_hf_135 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_hf_135
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_fol = ((end_y - start_y) - traj_curve.get_move_y()) * math.sqrt(2)
    d_pre = (end_x - start_x) - traj_curve.get_move_x() + d_fol/math.sqrt(2)
    
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.r_traj_x[-1] + d_pre , traj_curve.r_traj_x[-1] + d_pre - d_fol/math.sqrt(2) ],[traj_curve.r_traj_y[-1], traj_curve.r_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[-1] + d_pre , traj_curve.l_traj_x[-1] + d_pre - d_fol/math.sqrt(2) ],[traj_curve.l_traj_y[-1], traj_curve.l_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 

    
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [41]:
print(" ")
print("len(kappa) " + str(len(traj_hf_135.kappa)))
print("s resolution[m] " + str(traj_hf_135.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_hf_135.s, traj_hf_135.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 13999
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
0.0,
0.0,
2.4374708270599305e-291,
7.073700745982598e-233,
6.680070284029855e-194,
4.6148695896718563e-166,
3.496957594553924e-145,
6.071877293926994e-129,
5.95698885374758e-116,
2.538636604117903e-105,
1.8305295731211297e-96,
5.725133571785054e-89,
1.5214186367693143e-82,
5.624905457931898e-77,
4.1879064760607885e-72,
8.331339093147694e-68,
5.51818244264621e-64,
1.4476281194054448e-60,
1.7283461248866283e-57,
1.0508923226351301e-54,
3.567798793181228e-52,
7.297401799054823e-50,
9.580125827856058e-48,
8.513673868799769e-46,
5.357453344067319e-44,
2.4805696292177102e-42,
8.733185668547611e-41,
2.4050805936919242e-39,
5.309928546276154e-38,
9.601518900245197e-37,
1.4488128214146203e-35,
1.8545956379082885e-34,
2.0434025134884438e-33,
1.9629494413244787e-32,
1.662942618999976e-31,
1.255117397180721e-30,
8.517074661755674e-30,
5.239013290786658e-29,
2.9427950460412285e-28,
1.519527206020061e-27,
7.256153594230345e-27,
3.2219

9.184007488948035,
9.192982384930133,
9.20195462748376,
9.210924212621405,
9.2198911363715,
9.228855394778329,
9.237816983902013,
9.246775899818466,
9.255732138619326,
9.264685696411966,
9.273636569319388,
9.28258475348022,
9.291530245048675,
9.300473040194452,
9.309413135102783,
9.318350525974328,
9.327285209025128,
9.336217180486607,
9.345146436605503,
9.354072973643808,
9.362996787878766,
9.371917875602803,
9.380836233123503,
9.389751856763526,
9.398664742860637,
9.407574887767586,
9.416482287852135,
9.425386939496963,
9.434288839099652,
9.443187983072658,
9.452084367843225,
9.460977989853385,
9.469868845559919,
9.478756931434267,
9.48764224396256,
9.496524779645506,
9.505404534998407,
9.514281506551095,
9.523155690847865,
9.532027084447504,
9.540895683923166,
9.54976148586241,
9.558624486867101,
9.56748468355342,
9.576342072551762,
9.585196650506758,
9.594048414077205,
9.602897359936035,
9.61174348477027,
9.620586785280972,
9.629427258183233,
9.638264900206103,
9.647099708092593,
9

16.579469077110073,
16.58526736186617,
16.59106268493878,
16.596855047368376,
16.602644450195825,
16.60843089446239,
16.61421438120974,
16.61999491147993,
16.625772486315398,
16.631547106758955,
16.637318773853817,
16.643087488643566,
16.648853252172174,
16.654616065483967,
16.660375929623655,
16.666132845636312,
16.671886814567383,
16.67763783746268,
16.68338591536835,
16.689131049330932,
16.69487324039729,
16.700612489614677,
16.706348798030646,
16.712082166693143,
16.71781259665043,
16.723540088951125,
16.729264644644175,
16.734986264778872,
16.740704950404837,
16.74642070257202,
16.752133522330716,
16.757843410731514,
16.763550368825364,
16.7692543976635,
16.77495549829751,
16.780653671779273,
16.78634891916099,
16.792041241495163,
16.797730639834622,
16.80341711523249,
16.809100668742182,
16.814781301417437,
16.820459014312274,
16.826133808481014,
16.831805684978278,
16.83747464485896,
16.843140689178263,
16.848803818991655,
16.854464035354898,
16.86012133932404,
16.8657757319554,

21.069531318822992,
21.07288684522764,
21.07624040342281,
21.079591994246336,
21.082941618535656,
21.08628927712786,
21.089634970859642,
21.092978700567325,
21.096320467086855,
21.099660271253814,
21.10299811390339,
21.106333995870397,
21.109667917989295,
21.112999881094137,
21.116329886018626,
21.11965793359608,
21.12298402465943,
21.12630816004125,
21.129630340573726,
21.132950567088685,
21.136268840417557,
21.13958516139141,
21.142899530840932,
21.14621194959644,
21.149522418487884,
21.152830938344817,
21.15613750999643,
21.15944213427155,
21.16274481199861,
21.16604554400568,
21.169344331120453,
21.172641174170245,
21.175936073981998,
21.17922903138229,
21.18252004719732,
21.18580912225289,
21.18909625737447,
21.192381453387124,
21.195664711115555,
21.198946031384082,
21.202225415016674,
21.2055028628369,
21.208778375667972,
21.21205195433271,
21.21532359965359,
21.21859331245269,
21.221861093551716,
21.22512694377203,
21.228390863934585,
21.23165285485998,
21.234912917368433,
21.2

24.28996699887925,
24.291131191613225,
24.292294335477525,
24.293456430928412,
24.294617478422037,
24.295777478414436,
24.296936431361548,
24.298094337719174,
24.299251197943004,
24.30040701248863,
24.30156178181152,
24.302715506367036,
24.303868186610405,
24.305019822996776,
24.306170415981164,
24.30731996601847,
24.308468473563494,
24.309615939070913,
24.3107623629953,
24.311907745791117,
24.313052087912705,
24.314195389814305,
24.315337651950042,
24.31647887477392,
24.31761905873986,
24.31875820430165,
24.31989631191296,
24.321033382027377,
24.32216941509836,
24.323304411579254,
24.32443837192332,
24.325571296583675,
24.32670318601335,
24.327834040665266,
24.328963860992225,
24.33009264744693,
24.33122040048196,
24.33234712054981,
24.333472808102847,
24.334597463593337,
24.33572108747344,
24.336843680195212,
24.337965242210586,
24.339085773971405,
24.3402052759294,
24.34132374853619,
24.342441192243296,
24.343557607502124,
24.344672994763982,
24.34578735448006,
24.346900687101453,
2

...

24.54031557854632,
24.5393893211289,
24.53846212062416,
24.53753397659767,
24.536604888614924,
24.535674856241368,
24.53474387904235,
24.53381195658317,
24.532879088429034,
24.531945274145112,
24.531010513296472,
24.530074805448127,
24.52913815016501,
24.52820054701199,
24.52726199555386,
24.52632249535534,
24.525382045981093,
24.524440646995682,
24.52349829796363,
24.52255499844935,
24.521610748017224,
24.520665546231527,
24.519719392656473,
24.51877228685622,
24.517824228394815,
24.51687521683627,
24.5159252517445,
24.514974332683344,
24.51402245921659,
24.51306963090792,
24.51211584732098,
24.511161108019298,
24.510205412566346,
24.50924876052554,
24.508291151460195,
24.507332584933554,
24.506373060508793,
24.505412577749006,
24.504451136217217,
24.503488735476363,
24.502525375089313,
24.50156105461887,
24.500595773627722,
24.499629531678522,
24.498662328333825,
24.497694163156112,
24.496725035707787,
24.495754945551177,
24.494783892248517,
24.493811875361992,
24.492838894453683,
24

21.820265515506858,
21.81735739623898,
21.814447497758895,
21.811535819315154,
21.80862236015593,
21.80570711952907,
21.802790096682063,
21.79987129086206,
21.796950701315847,
21.794028327289865,
21.791104168030202,
21.78817822278261,
21.785250490792464,
21.782320971304816,
21.77938966356436,
21.776456566815416,
21.773521680301982,
21.77058500326769,
21.76764653495583,
21.76470627460934,
21.761764221470774,
21.758820374782385,
21.755874733786047,
21.752927297723282,
21.749978065835254,
21.7470270373628,
21.744074211546373,
21.7411195876261,
21.738163164841723,
21.73520494243268,
21.732244919638003,
21.7292830956964,
21.72631946984623,
21.723354041325475,
21.720386809371785,
21.71741777322244,
21.714446932114388,
21.711474285284194,
21.708499831968087,
21.70552357140194,
21.70254550282127,
21.69956562546124,
21.696583938556643,
21.693600441341957,
21.69061513305126,
21.68762801291828,
21.68463908017644,
21.681648334058735,
21.67865577379786,
21.67566139862613,
21.672665207775495,
21.669

16.18418317262524,
16.17818107713016,
16.172175949291873,
16.16616778810533,
16.16015659256612,
16.15414236167046,
16.148125094415146,
16.142104789797663,
16.13608144681612,
16.130055064469236,
16.124025641756393,
16.11799317767763,
16.111957671233604,
16.105919121425657,
16.09987752725578,
16.093832887726602,
16.08778520184144,
16.081734468604267,
16.075680687019734,
16.06962385609317,
16.06356397483055,
16.057501042238577,
16.051435057324625,
16.045366019096726,
16.039293926563644,
16.033218778734835,
16.02714057462042,
16.021059313231284,
16.01497499357895,
16.0088876146757,
16.00279717553454,
15.99670367516913,
15.990607112593947,
15.984507486824112,
15.978404796875513,
15.972299041764769,
15.96619022050924,
15.960078332127019,
15.953963375636949,
15.947845350058621,
15.941724254412376,
15.935600087719337,
15.929472849001328,
15.92334253728102,
15.917209151581796,
15.911072690927808,
15.904933154344022,
15.898790540856165,
15.892644849490729,
15.886496079275037,
15.880344229237176,

11.254701910405565,
11.24644401327205,
11.238182878169303,
11.229918506328985,
11.22165089899083,
11.213380057402727,
11.20510598282073,
11.196828676509037,
11.188548139740057,
11.18026437379448,
11.171977379961191,
11.163687159537343,
11.155393713828468,
11.147097044148344,
11.138797151819153,
11.130494038171406,
11.122187704544057,
11.113878152284482,
11.105565382748473,
11.097249397300324,
11.088930197312846,
11.080607784167343,
11.072282159253655,
11.063953323970283,
11.055621279724232,
11.047286027931193,
11.03894757001548,
11.030605907410102,
11.02226104155678,
11.013912973905926,
11.005561705916746,
10.997207239057229,
10.988849574804142,
10.980488714643064,
10.972124660068522,
10.963757412583833,
10.955386973701282,
10.947013344942086,
10.938636527836382,
10.93025652392336,
10.921873334751158,
10.913486961877013,
10.90509740686723,
10.896704671297153,
10.888308756751304,
10.879909664823353,
10.871507397116106,
10.863101955241598,
10.854693340821116,
10.84628155548515,
10.837866

0.03643006686272202,
0.03573101537816908,
0.035041712222218746,
0.034362083001315706,
0.033692053145087796,
0.03303154791257635,
0.03238049239856458,
0.03173881153996001,
0.03110643012223654,
0.03048327278597419,
0.029869264033440507,
0.029264328235257684,
0.02866838963713397,
0.02808137236664591,
0.02750320044011647,
0.026933797769537898,
0.02637308816955459,
0.02582099536453206,
0.02527744299567359,
0.024742354628192385,
0.024215653758567202,
0.023697263821835752,
0.02318710819896247,
0.022685110224260147,
0.02219119319285764,
0.021705280368245213,
0.021227294989860562,
0.020757160280722282,
0.02029479945513488,
0.019840135726432843,
0.01939309231476728,
0.01895359245496041,
0.018521559404394226,
0.01809691645093827,
0.017679586920944936,
0.017269494187257164,
0.016866561677284078,
0.01647071288110144,
0.01608187135958585,
0.015699960752601272,
0.015324904787207315,
0.014956627285896721,
0.014595052174877643,
0.014240103492374841,
0.01389170539695206,
0.013549782175878019,
0.01321425

## クラシック 斜め 90度ターン

In [45]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.045
start_y = 0.00
end_x = -0.045
end_y = 0.09
target_ang = 90
start_ang = 45

shape_factor__ = 2.6
path_length__ = 0.09
v__ = 0.5
traj_hf_D_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_hf_D_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_fol = ((end_y - start_y) - traj_curve.get_move_y()) /(math.sqrt(2))
    d_pre = d_fol
    
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    #plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([traj_curve.r_traj_x[0]  ,traj_curve.r_traj_x[0] + d_pre/math.sqrt(2)],[traj_curve.r_traj_y[0], traj_curve.r_traj_y[0] + d_pre/math.sqrt(2)]  ,color="red") 
    plt.plot([start_x,traj_curve.c_traj_x[0] + d_pre/math.sqrt(2)],[start_y, traj_curve.c_traj_y[0] + d_pre/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[0], traj_curve.l_traj_x[0] + d_pre/math.sqrt(2)],[traj_curve.l_traj_y[0], traj_curve.l_traj_y[0] + d_pre/math.sqrt(2)]  ,color="red") 

    #plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x + d_pre/math.sqrt(2), traj_curve.r_traj_y + d_pre/math.sqrt(2), color="blue")
    plt.plot(traj_curve.c_traj_x + d_pre/math.sqrt(2), traj_curve.c_traj_y + d_pre/math.sqrt(2) , color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre/math.sqrt(2), traj_curve.l_traj_y + d_pre/math.sqrt(2), color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.r_traj_x[-1] + d_pre/math.sqrt(2) , traj_curve.r_traj_x[-1]  ],[traj_curve.r_traj_y[-1]+ d_pre/math.sqrt(2), traj_curve.r_traj_y[-1]+ 2*d_fol/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.get_end_x() + d_pre/math.sqrt(2) , end_x],[traj_curve.get_end_y() + d_pre/math.sqrt(2), end_y]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[-1] + d_pre/math.sqrt(2) , traj_curve.l_traj_x[-1]  ],[traj_curve.l_traj_y[-1]+ d_pre/math.sqrt(2) , traj_curve.l_traj_y[-1]+ 2*d_fol/math.sqrt(2)]  ,color="red") 

    
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [43]:
print(" ")
print("len(kappa) " + str(len(traj_hf_D_90.kappa)))
print("s resolution[m] " + str(traj_hf_D_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_hf_D_90.s, traj_hf_D_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 7000
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
3.249961102746574e-291,
8.906760378706474e-194,
4.662610126071899e-145,
7.942651804996774e-116,
2.4407060974948397e-96,
2.028558182359086e-82,
5.583875301414385e-72,
7.357576590194948e-64,
2.304461499848838e-57,
4.7570650575749715e-52,
1.277350110380808e-47,
7.143271125423093e-44,
1.1644247558063483e-40,
7.079904728368206e-38,
1.9317504285528273e-35,
2.7245366846512587e-33,
2.2172568253333016e-31,
1.1356099549007567e-29,
3.9237267280549716e-28,
9.674871458973793e-27,
1.78257974174235e-25,
2.549268671885519e-24,
2.920786212029926e-23,
2.7533114840531263e-22,
2.184030568068072e-21,
1.4860652930445223e-20,
8.81721006708503e-20,
4.626923437248133e-19,
2.1739746914076106e-18,
9.244060079673658e-18,
3.590720216070573e-17,
1.2846436524244042e-16,
4.264004418336144e-16,
1.3215325411764793e-15,
3.8463123647987446e-15,
1.056648893176057e-14,
2.7524304753868926e-14,
6.826191658206114e-14,
1.6177683215975966e-13,
3.6759800384480993e-

25.637982432591265,
25.649703865125247,
25.66141244695027,
25.673108188852563,
25.684791101611683,
25.696461196000474,
25.708118482785082,
25.719762972724894,
25.731394676572556,
25.74301360507394,
25.754619768968126,
25.766213178987364,
25.777793845857104,
25.78936178029595,
25.800916993015612,
25.81245949472094,
25.823989296109897,
25.835506407873503,
25.84701084069587,
25.85850260525413,
25.86998171221849,
25.88144817225213,
25.892901996011254,
25.904343194145056,
25.915771777295685,
25.927187756098256,
25.938591141180787,
25.94998194316427,
25.961360172662545,
25.972725840282383,
25.984078956623417,
25.995419532278145,
26.006747577831884,
26.018063103862808,
26.02936612094191,
26.04065663963296,
26.051934670492525,
26.063200224069952,
26.074453310907348,
26.085693941539542,
26.0969221264941,
26.108137876291334,
26.119341201444232,
26.130532112458475,
26.141710619832427,
26.152876734057106,
26.164030465616204,
26.175171824986013,
26.18630082263549,
26.197417469026185,
26.20852177461

32.41892718641907,
32.4219718330319,
32.42501093906887,
32.42804450936984,
32.43107254877234,
32.434095062111574,
32.43711205422036,
32.44012352992924,
32.44312949406641,
32.446129951457785,
32.449124906926954,
32.45211436529521,
32.45509833138159,
32.45807681000284,
32.46104980597342,
32.46401732410554,
32.46697936920917,
32.46993594609203,
32.47288705955959,
32.475832714415105,
32.4787729154596,
32.48170766749189,
32.48463697530859,
32.48756084370411,
32.49047927747068,
32.49339228139834,
32.49629986027499,
32.499202018886294,
32.502098762015834,
32.504990094445,
32.50787602095306,
32.51075654631714,
32.513631675312254,
32.51650141271129,
32.51936576328502,
32.522224731802126,
32.525078323029206,
32.52792654173076,
32.53076939266921,
32.53360688060493,
32.5364390102962,
32.539265786499286,
32.54208721396839,
32.54490329745566,
32.547714041711274,
32.550519451483325,
32.55331953151792,
32.556114286559186,
32.558903721349225,
32.56168784062815,
32.56446664913412,
32.5672401516033,
32.5

33.48355757947454,
33.48350273526301,
33.483446739409644,
33.483389582958566,
33.483331256999556,
33.483271752667115,
33.483211061139805,
33.48314917363938,
33.48308608143013,
33.48302177581809,
33.48295624815038,
33.482889489814504,
33.48282149223772,
33.48275224688634,
33.48268174526516,
33.48260997891683,
33.48253693942122,
33.48246261839492,
33.48238700749063,
33.482310098396574,
33.48223188283607,
33.482152352566914,
33.482071499380886,
33.48198931510331,
33.481905791592524,
33.481820920739395,
33.48173469446689,
33.48164710472963,
33.48155814351342,
33.481467802834814,
33.48137607474078,
33.481282951308145,
33.48118842464334,
33.48109248688191,
33.48099513018817,
33.48089634675483,
33.48079612880261,
33.4806944685799,
33.48059135836238,
33.480486790452744,
33.480380757180264,
33.480273250900545,
33.480164263995135,
33.48005378887128,
33.47994181796157,
33.47982834372359,
33.479713358639735,
33.479596855216805,
33.47947882598582,
33.47935926350163,
33.479238160342724,
33.479115509

29.011760499709844,
29.003903063631032,
28.996036049817068,
28.98815945016813,
28.9802732565769,
28.972377460928534,
28.964472055100632,
28.956557030963257,
28.948632380378918,
28.94069809520259,
28.932754167281658,
28.92480058845594,
28.91683735055771,
28.90886444541165,
28.90088186483484,
28.89288960063677,
28.884887644619358,
28.876875988576906,
28.86885462429609,
28.860823543556,
28.85278273812808,
28.84473219977617,
28.83667192025645,
28.828601891317494,
28.82052210470021,
28.81243255213786,
28.804333225356057,
28.796224116072736,
28.788105215998215,
28.779976516835067,
28.771838010278255,
28.76368968801501,
28.755531541724906,
28.747363563079794,
28.739185743743846,
28.73099807537354,
28.722800549617617,
28.71459315811711,
28.706375892505342,
28.6981487444079,
28.689911705442626,
28.681664767219665,
28.673407921341365,
28.665141159402374,
28.65686447298956,
28.648577853682045,
28.640281293051185,
28.631974782660553,
28.62365831406597,
28.615331878815468,
28.606995468449302,
28.59

14.763042879004875,
14.740828372965643,
14.718596760020642,
14.696348055409027,
14.674082274555651,
14.651799433072167,
14.629499546758009,
14.607182631601711,
14.584848703781843,
14.562497779668211,
14.540129875822956,
14.517745009001723,
14.49534319615481,
14.472924454428135,
14.450488801164695,
14.428036253905423,
14.405566830390592,
14.383080548560743,
14.36057742655809,
14.338057482727546,
14.31552073561796,
14.292967203983343,
14.270396906783967,
14.247809863187735,
14.225206092571145,
14.202585614520798,
14.179948448834386,
14.157294615522085,
14.134624134807604,
14.111937027129656,
14.089233313143094,
14.066513013720046,
14.043776149951439,
14.021022743148038,
13.998252814841878,
13.97546638678738,
13.952663480962846,
13.929844119571568,
13.907008325043268,
13.8841561200353,
13.861287527433998,
13.838402570356102,
13.81550127214987,
13.792583656396642,
13.769649746912016,
13.746699567747331,
13.723733143190877,
13.700750497769356,
13.677751656249285,
13.654736643638257,
13.6317